In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [2]:
folder="/home/spoudel/All_Thesis/Thesis_program/Data/shawnee/location_1/Kmeans/"

In [3]:
location_in_time=pd.read_csv(folder+"/location_in_time.csv")

In [4]:
location_in_time

,Unnamed: 0,Start,End,passID,lat_min,lat_max,long_min,long_max
0,0,2023-01-16T17:24:39.000Z,2023-01-16T17:25:25.000Z,115,35.670006,35.670785,-96.662316,-96.662305
1,1,2023-01-16T17:25:52.000Z,2023-01-16T17:26:38.000Z,117,35.670014,35.670792,-96.662317,-96.662304
2,2,2023-01-16T17:27:08.000Z,2023-01-16T17:27:55.000Z,118,35.670003,35.670800,-96.662341,-96.662327
3,3,2023-01-16T17:29:06.000Z,2023-01-16T17:29:53.000Z,120,35.670000,35.670794,-96.662342,-96.662329
4,4,2023-01-16T17:30:23.000Z,2023-01-16T17:31:09.000Z,121,35.670007,35.670784,-96.662333,-96.662319


In [5]:
passes=location_in_time["passID"].values
passes

array([115, 117, 118, 120, 121])

In [6]:
each_data=[pd.read_csv(folder+f"/cluster_predicted_pass_{x}.csv") for x in location_in_time["passID"]]

In [7]:
# Compute heading
def compute_heading(df):
    df["heading"] = 0
    for i in range(len(df) - 1):
        a = df.loc[i, "latitude"] - df.loc[i + 1, "latitude"]
        b = df.loc[i, "longitude"] - df.loc[i + 1, "longitude"]
        d = 180 / np.pi * np.arctan2(a, b) + 180
        df.loc[i, "heading"] = d

In [8]:
def projectRollerWidth(df_in, rollerWidth=6, nPoints=6):
    df=df_in.copy(deep=True)
    copyDF = df.copy()
    halfWidth = rollerWidth // 2
    for i in np.arange(-halfWidth, halfWidth, rollerWidth / nPoints):
        tmpDF = copyDF.copy()
        tmpDF.latitude = (
            np.sin(2 * np.pi / 360 * (copyDF.heading.values - 90)) * i / 3600 / 101
            + copyDF.latitude.values
        )
        tmpDF.longitude = (
            np.cos(2 * np.pi / 360 * (copyDF.heading.values - 90)) * i / 3600 / 80
            + copyDF.longitude.values
        )
        tmpDF.original = False
        df = df.append(tmpDF)
    df =df.sort_values("timestamp")
    return df

In [9]:
each_data_projected=[[]]*len(each_data)
for i in range(len(passes)):
    each_data[i]["pass_no"]=i
    length=len(each_data[i])
    latitudes = np.linspace(location_in_time["lat_min"][i],location_in_time["lat_max"][i] , num=length)
    print(len(latitudes))
    each_data[i]["latitude"]=latitudes
    longitudes= np.linspace(location_in_time["long_min"][i],location_in_time["long_max"][i] , num=length)
    each_data[i]["longitude"]=longitudes
    
    each_data[i]["prev"]=each_data[i]["cluster"].shift(1)
    each_data[i]["change"]=each_data[i].apply(lambda x: 1 if abs(x["prev"]-x["cluster"]>1)  else 0, axis=1)
       
    print(i)
    if i==0:
        each_data[i]["first"]=True
        each_data[i]["last"]=False
    elif i==len(passes)-1:
        each_data[i]["first"]=False
        each_data[i]["last"]=True
    else:
        each_data[i]["first"]=False
        each_data[i]["last"]=False
        
        
    compute_heading(each_data[i])
    each_data_projected[i]=projectRollerWidth(each_data[i])

        
full_data=pd.concat(each_data)
full_data=full_data[full_data["pass_no"]!=0]

full_data_projected=pd.concat(each_data_projected)
full_data_projected=full_data_projected[full_data_projected["pass_no"]!=0]

358
0
360
1
366
2


/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_

366
3
360
4


/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)
/tmp/ipykernel_497128/2125603734.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmpDF)


In [10]:
full_data["time"]=full_data["timestamp"]
full_data["passID"]=full_data["pass_no"]
full_data["speed"]=1.1
full_data["temperature"]=167.5

full_data["density"]=full_data["cluster"]
full_data.to_csv(folder+"/full_data_to_project.csv")
full_data

,Unnamed: 0.1,Unnamed: 0,timestamp,total_power,p0,p1,p2,p3,p4,p5,...,prev,change,first,last,heading,time,passID,speed,temperature,density
0,0,0,2023-01-16 17:25:52.120000+00:00,100.000000,-2.000000,2.000000,2.000000,0.935329,2.000000,2.000000,...,NaN,0,False,False,89.042705,2023-01-16 17:25:52.120000+00:00,1,1.1,167.5,4
1,1,1,2023-01-16 17:25:52.257000+00:00,82.183425,-2.000000,-0.085897,-0.100313,0.562901,1.384732,0.605973,...,4.0,0,False,False,89.042705,2023-01-16 17:25:52.257000+00:00,1,1.1,167.5,3
2,2,2,2023-01-16 17:25:52.384000+00:00,83.212034,-2.000000,0.909739,-0.090134,0.542970,0.487589,0.860677,...,3.0,1,False,False,89.042705,2023-01-16 17:25:52.384000+00:00,1,1.1,167.5,0
3,3,3,2023-01-16 17:25:52.511000+00:00,58.360112,-0.510243,-0.707589,-0.276745,-0.355957,-0.028058,0.654395,...,0.0,0,False,False,89.042705,2023-01-16 17:25:52.511000+00:00,1,1.1,167.5,2
4,4,4,2023-01-16 17:25:52.638000+00:00,62.575446,0.922105,-0.875624,-0.435802,0.125723,-0.576983,0.532818,...,2.0,0,False,False,89.042705,2023-01-16 17:25:52.638000+00:00,1,1.1,167.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,355,355,2023-01-16 17:31:08.215000+00:00,18.406951,0.619154,2.000000,-0.067383,-0.711903,-0.299709,-0.395371,...,4.0,1,False,True,88.967755,2023-01-16 17:31:08.215000+00:00,4,1.1,167.5,2
356,356,356,2023-01-16 17:31:08.342000+00:00,67.584552,0.658804,0.218019,-0.212631,-0.182176,1.719183,1.880953,...,2.0,0,False,True,88.967755,2023-01-16 17:31:08.342000+00:00,4,1.1,167.5,4
357,357,357,2023-01-16 17:31:08.469000+00:00,82.175854,0.966712,0.201359,0.004933,2.000000,0.814174,-0.509651,...,4.0,0,False,True,88.967755,2023-01-16 17:31:08.469000+00:00,4,1.1,167.5,3
358,358,358,2023-01-16 17:31:08.596000+00:00,100.000000,1.250349,-0.648439,1.299200,2.000000,-0.037076,-0.079023,...,3.0,0,False,True,88.967755,2023-01-16 17:31:08.596000+00:00,4,1.1,167.5,3


In [90]:
# full_data_projected["time"]=full_data_projected["timestamp"]
# full_data_projected["passID"]=full_data_projected["pass_no"]

# full_data_projected["speed"]=1.1
# full_data_projected["temperature"]=167.5

# full_data_projected["density"]=full_data_projected[""]

# full_data_projected.to_csv(folder+"full_data_projected.csv")

In [38]:
pd.unique(full_data_projected[['latitude', 'longitude']].values.ravel('K'))

21676

In [45]:
# Determine number of x bins for coverage plot
lon_min = full_data_projected.longitude.min()
lon_max = full_data_projected.longitude.max()
lat_min = full_data_projected.longitude.min()
lat_max = full_data_projected.longitude.max()
nHexBins_lat = int(
    (lat_max - lat_min) * 3600 * 80 / 2
)  
nHexBins_lon = int(
    (lon_max - lon_min) * 3600 * 80 / 2
)  # convert longitude to feet: hex bin width == 2 ft

In [48]:
nHexBins=max(nHexBins_lon,nHexBins_lat)
nHexBins

7

In [50]:
def getLastValue(hexBin):
        return hexBin[-1]

In [53]:
coverageFig = {}
# Coverage map colored by pass count
coverageFig["passes"] = ff.create_hexbin_mapbox(
    data_frame=full_data_projected,
    lat="latitude",
    lon="longitude",
    labels={"color": "Number of Passes"},
    nx_hexagon=nHexBins,
    agg_func=getLastValue,
    color="pass_no",
    zoom=18,
)

In [80]:
p=coverageFig["passes"]["data"][0]
p

Choroplethmapbox({
    'coloraxis': 'coloraxis',
    'customdata': array([[3.0, '-1.6870763123695685,0.6671710130044683', 0],
                         [3.0, '-1.6870763123695685,0.6671712519187006', 0],
                         [3.0, '-1.6870763123695685,0.6671714908329329', 0],
                         ...,
                         [1.0, '-1.6870754157777923,0.6671876175436132', 0],
                         [1.0, '-1.6870754157777923,0.6671878564578455', 0],
                         [1.0, '-1.6870754157777923,0.6671880953720778', 0]], dtype=object),
    'geojson': {'features': [{'geometry': {'coordinates': [[[-96.66235241527079,
                                                           35.669994473247655],
                                                           [-96.6623484636612,
                                                           35.66999632668299],
                                                           [-96.6623484636612,
                                             

In [88]:
p.customdata

array([[3.0, '-1.6870763123695685,0.6671710130044683', 0],
       [3.0, '-1.6870763123695685,0.6671712519187006', 0],
       [3.0, '-1.6870763123695685,0.6671714908329329', 0],
       ...,
       [1.0, '-1.6870754157777923,0.6671876175436132', 0],
       [1.0, '-1.6870754157777923,0.6671878564578455', 0],
       [1.0, '-1.6870754157777923,0.6671880953720778', 0]], dtype=object)